In [1]:
#!git clone https://github.com/starbucksdolcelatte/ShowMeTheColor

In [2]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing.image import  ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import  Dense,Flatten,Input
import os
from tensorflow.keras.callbacks import Callback

In [3]:
size=(224,224,3)
size

(224, 224, 3)

In [4]:
image = ImageDataGenerator(
    rescale=1/255,
    rotation_range=30,       # 회전 범위 확대
    width_shift_range=0.3,   # 수평 이동 확대
    height_shift_range=0.3,  # 수직 이동 확대
    shear_range=0.3,         # 전단 범위 확대
    zoom_range=0.3,          # 줌 범위 확대
    horizontal_flip=True,
    brightness_range=[0.8, 1.2],  # 밝기 조정
    fill_mode="nearest"
)


In [5]:
train=image.flow_from_directory(os.path.join('img','train'),target_size=size[:2],batch_size=32,shuffle=True)
test=image.flow_from_directory(os.path.join('img','test'),target_size=size[:2],batch_size=32,shuffle=True)

Found 213 images belonging to 4 classes.
Found 40 images belonging to 4 classes.


In [6]:
vgg16=VGG16(include_top=False,input_shape=size)
vgg16.summary()

for layer in vgg16.layers:
    layer.trainable = False

Model: "vgg16"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 224, 224, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 224, 224, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 112, 112, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 112, 112, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 56, 56, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 28, 28, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 28, 28, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 14, 14, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 7, 7, 512)      │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,714,688 (56.13 MB)

 Trainable params: 14,714,688 (56.13 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:

# Sequential 모델 생성
# - Input: 입력 레이어 (224x224x3 크기의 이미지)
# - vgg16: 전이학습을 위한 VGG16 모델
# - Flatten: 특성 맵을 1차원으로 펼침
# - Dense(64): 64개 뉴런을 가진 완전연결층, ReLU 활성화 함수 사용
# - Dense(4): 출력층, 클래스 수만큼의 뉴런, softmax 활성화 함수로 확률 출력
model=Sequential((
    Input(shape=size),
    vgg16,Flatten(),
    Dense(64,activation="relu"),
    Dense(len(train.class_indices),activation="softmax")))

# 모델 컴파일
# - optimizer: Adam 옵티마이저 사용
# - loss: 다중 클래스 분류를 위한 categorical_crossentropy
# - metrics: 정확도(accuracy) 측정
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 모델 구조 출력
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ vgg16 (Functional)              │ (None, 7, 7, 512)      │    14,714,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │     1,605,696 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 4)              │           260 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 16,320,644 (62.26 MB)

 Trainable params: 1,605,956 (6.13 MB)

 Non-trainable params: 14,714,688 (56.13 MB)

In [8]:
train.class_indices

{'fall': 0, 'spring': 1, 'summer': 2, 'winter': 3}

In [19]:
with open("labels.txt",'w') as f:
    f.write("\n".join(['가을 웜톤','봄 웜톤','여름 쿨톤','겨울 쿨톤']))    

In [11]:
class TerminateOnThreshold(Callback):
    def __init__(self, accuracy,val_accuracy):
        super(TerminateOnThreshold, self).__init__()
        self.accuracy = accuracy
        self.val_accuracy = val_accuracy

    def on_epoch_end(self, epoch, logs=None):
        val_accuracy = logs.get("val_accuracy")  # 정확도 확인
        accuracy = logs.get("accuracy") 
        if accuracy and val_accuracy and accuracy >= self.accuracy and val_accuracy >= self.val_accuracy:
            print(f"\nEpoch {epoch+1}: Reached accuracy {accuracy:.4f} (>= {self.accuracy}), val_accuracy {val_accuracy:.4f} (>= {self.val_accuracy}), stopping training!")
            self.model.stop_training = True

# 사용자 정의 콜백 설정
terminate_on_threshold = TerminateOnThreshold(accuracy=0.9,val_accuracy=0.8)
model.fit(train, epochs=1, validation_data=test, callbacks=[terminate_on_threshold])


7/7 ━━━━━━━━━━━━━━━━━━━━ 18s 2s/step - accuracy: 0.9988 - loss: 0.0041 - val_accuracy: 0.3250 - val_loss: 3.4412


In [ ]:
# 모델 평가
loss, accuracy = model.evaluate(test)
print(f"Test Accuracy: {(accuracy)*100}")

In [ ]:
model.save('model.keras')